#CMTH642 - Data Analytics: Advanced Methods
#Assignment 2 (10%)
Ann Sam

Section DHA - Student #501160843

Due on June 6, 20222 11:30 PM

Question 1: 
Import the following file: http://archive.ics.uci.edu/ml/machine-learning-databases/winequality/winequality-white.csv

Check the datatypes of the attributes (2 points)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [3]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("winequality-white.csv", sep=";")
df = pd.DataFrame(data)
print(df.dtypes)

In [ ]:
df.describe()

Question 2: Are there any missing values in the dataset? (2 points)

*Using df.info() we can see that there are 4898 entries for each column and see that every column contains the correct amount non-null entries. We can also use df.isna().any().any() to return a boolean result if there are any NA values and the same with df.isnull().any().any(). The output is FALSE.*

In [ ]:
df.info()
df.isna().any().any()
df.isnull().any().any()

Question 3: What is the correlation between the attributes other than Quality? (8
points)

*After analyzing the correlation matrix of the attributes (not including Quality) and the heat map to visualize the correlations, we can see immediately that there are correlations between some of the attributes.*


In [ ]:
pred_var = df.drop("quality", axis=1)
corrMatrix = pred_var.corr()
print(corrMatrix)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

sn.heatmap(corrMatrix, cmap='coolwarm', annot=True, fmt=".2f")

plt.xticks(range(len(corrMatrix.columns)), corrMatrix.columns);
plt.yticks(range(len(corrMatrix.columns)), corrMatrix.columns)

plt.show()

*There are strong correlations between:*

1.   Density and Residual Sugars (+) 0.84
2.   Density and Alcohol (-) 0.78
3.   Total Sulfer Dioxide and Free Sulfur Dioxide (+) 0.62

*There are moderate correlations between:*

1.   Density and Total Sulfur Dioxide (+) 0.53
2.   Alcohol and Total Sulfur Dioxide (-) 0.45
3.   Alcohol and Residual Sugars (-) 0.45
4.   Residual Sugars and Total Sulfur Dioxide (+) 0.40

Question 4: Graph the frequency distribution of wine quality by using Quality. (8
points)

In [ ]:
sn.countplot(x='quality', data=df, palette='muted')

Question 5. Reduce the levels of rating for quality to two levels as Pass and Fail.
Assign the levels of 3, 4 and 5 to level Fail; and 6, 7, 8 and 9 to level Pass.
(10 points)

In [8]:
df['quality'] = df.quality.apply(lambda q: 'Pass' if q <= 5 else 'Fail')

In [ ]:
df.quality.value_counts()

In [ ]:
df.head()

Question 6. Normalize the numeric attributes. Hint: (x - min(x)) / (max(x) - min(x))
(10 points)

In [10]:
attributeColList = ['fixed acidity',	'volatile acidity',	'citric acid',	'residual sugar',	'chlorides',
                    'free sulfur dioxide',	'total sulfur dioxide',	'density',	'pH',	'sulphates',	'alcohol']

def normalize(data_set):
    result = df.copy()
    for attributeCol, i in zip(attributeColList, range(len(attributeColList))):
        max_value = df[attributeCol].max()
        min_value = df[attributeCol].min()
        result[attributeCol] = (df[attributeCol] - min_value) / (max_value - min_value)
    return result

In [ ]:
df2 = normalize(df)
df2.head()

In [ ]:
df2.describe()

Question 7: Prepare an exploratory data analysis question (without using predictions) about the dataset. Write a code to answer your question. Visualize your answer. (Your question should be related to at least three attributes in the dataset) (12 points)

*EDA Question: Deal with any outliers in the dataset and determine correlation between the attributes and the class variable Quality*

In [ ]:
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(30,10))
index = 0
ax = ax.flatten()

for attributeCol, value in df2.items():
    sn.boxplot(y=attributeCol, data=df2, ax=ax[index], color='steelblue')
    index += 1
plt.tight_layout(pad=1.5, w_pad=1.5, h_pad=5.0)

#error in plotting is due to 'quality' column having nonnumeric values. Will deal with this later.

*Analysis of the boxplots show that in every attribute there are outliers except for in alcohol. We can create a copy of the data frame where we remove all the outliers and use that to compare if the removal of outliers significantly effect our prediction models.*

In [13]:
df3 =df2.copy()
df3['quality'] = df3.quality.apply(lambda q: 1 if q == 'Pass' else 0)
#Changing Pass to 1 and Fail to 0 in the Quality column

In [14]:
def remove_outliers(dataset):
  dataset1 = dataset.copy()
  dataset = dataset._get_numeric_data()
  q1 = dataset.quantile(0.25)
  q3 = dataset.quantile(0.75)

  iqr = q3 - q1
  lowerBound = q1 -(1.5 * iqr) 
  upperBound = q3 +(1.5 * iqr)

  for col in dataset.columns:
    for i in range(0,len(dataset[col])):
      if dataset[col][i] < lowerBound[col]:
        dataset[col][i] = lowerBound[col]
      if dataset[col][i] > upperBound[col]:
        dataset[col][i] = upperBound[col]
    
  for col in dataset.columns:
    dataset1[col] = dataset[col]

  return(dataset1)

In [ ]:
df3 = remove_outliers(df3)

In [ ]:
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(30,10))
index = 0
ax = ax.flatten()

for attributeCol, value in df3.items():
    sn.boxplot(y=attributeCol, data=df3, ax=ax[index], color='seagreen')
    index += 1
plt.tight_layout(pad=1.5, w_pad=1.5, h_pad=5.0)

In [ ]:
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(20,10))
index = 0
ax = ax.flatten()

for col, value in df3.items():
    sn.distplot(value, color='seagreen', ax=ax[index])
    index += 1
plt.tight_layout(pad=0.5, w_pad=0.7, h_pad=5.0)

*Analysis of the distribution plots after removing outliers of all the continuous attributes, we can see that the attributes all still maintain a relative normal distibution with some attributes slightly right-skewed.*

In [ ]:
corrMatrix3 = df3.corr()

fig, ax = plt.subplots(figsize=(10, 8))

sn.heatmap(corrMatrix3, cmap='crest', annot=True, fmt=".2f")

plt.xticks(range(len(corrMatrix3.columns)), corrMatrix3.columns);
plt.yticks(range(len(corrMatrix3.columns)), corrMatrix3.columns)

plt.show()

*Analysisng the correlation matrix, we can see that Alcohol has the strongest correlation with Quality. We can also note that even after removing the outliers, we do still see similar correlation between other attributes as seen in question 3. We also see a few more moderately correlated attributes.*

*There are strong correlations between:*

1.   Density and Residual Sugars (+) 0.83
2.   Density and Alcohol (-) 0.80
3.   Total Sulfer Dioxide and Free Sulfur Dioxide (+) 0.62

*There are moderate correlations between:*

1.   Density and Total Sulfur Dioxide (+) 0.55
2.   Alcohol and Total Sulfur Dioxide (-) 0.45
3.   Alcohol and Residual Sugars (-) 0.45
4.   Residual Sugars and Total Sulfur Dioxide (+) 0.41
5.   Density and Chlorides (+) 0.46
6.   Alcohol and Chlorides (-) 0.46 
7.   Fixed Acidity and pH (-) 0.43

In [20]:
def FunctionAnova(inpData, TargetVariable, attributeList):
    from scipy.stats import f_oneway

    SelectedPredictors=[]
    
    for predictor in attributeList:
        CategoryGroupLists=inpData.groupby(TargetVariable)[predictor].apply(list)
        AnovaResults = f_oneway(*CategoryGroupLists)
        
        # If p-value <0.05, that means we reject H0
        if (AnovaResults[1] < 0.05):
            print(predictor, 'is correlated with', TargetVariable, ', P-Value:', AnovaResults[1])
            SelectedPredictors.append(predictor)
        else:
            print(predictor, 'is NOT correlated with', TargetVariable, ', P-Value:', AnovaResults[1])
    
    return(SelectedPredictors)

In [ ]:
FunctionAnova(inpData=df3, TargetVariable='quality', attributeList=attributeColList)

*After performing a one-way ANOVA test on the attributes with the class attribute Quality, we can see that two attributes are not correlated with Wine Quality: Citric Acid and Free Sulfur Dioxide. We can remove these attributes if we wanted to further test the performance of our model.*

Question 8: Divide the dataset to training and test sets. (12 points)

*Will use df2 to complete the remainder of the assignment (dataset without outlier removal)*

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

In [23]:
df2['quality'] = df2.quality.apply(lambda q: 1 if q == 'Pass' else 0)
#Changing Pass to 1 and Fail to 0 in the Quality column

X = df2[attributeColList]
y = df2['quality'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Question 9: Use the Logistic Regression algorithm to predict the quality of wine using its attributes. (12 points)

In [ ]:
classifier = LogisticRegression(random_state = 42)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

print(y_pred)

Question 10: Display the confusion matrix to evaluate the model performance. (12 points)

In [ ]:
confusion_mat = confusion_matrix(y_test, y_pred)
print(confusion_mat)

Question 11: Evaluate the model performance by computing Accuracy, Sensitivity and Specificity. (12 points)

In [ ]:
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = pd.DataFrame([['Logistic Regression', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
print(results)

***The following code is just to run the filtered dataset df3 with the outliers removed and the non-correlated attributes removed. We can see that accuarcy increased from 0.73 to 0.75, precision, reacll and the F-sore remained the same.*

In [ ]:
df3.drop(['citric acid', 'free sulfur dioxide'], axis = 1)

X = df3[attributeColList]
y = df3['quality'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

classifier = LogisticRegression(random_state = 42)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = pd.DataFrame([['Logistic Regression', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
print(results)